<a href="https://colab.research.google.com/github/regenbuild/BIM2BEM/blob/main/CellsFromVerticesAndFaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scipy
!pip install topologicpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.1/366.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.3 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3568 sha256=4196d521d3b8e063d69f93d85e0f18cd41ad5580da49818dedf0c5134ed4888e
  Stored i

In [2]:
from scipy.spatial import ConvexHull, Delaunay, Voronoi, delaunay_plot_2d, convex_hull_plot_2d, voronoi_plot_2d, KDTree
import numpy as np
import pandas as pd
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Topology import Topology
from topologicpy.Wire import Wire
from topologicpy.Cell import Cell
from topologicpy.Face import Face

import csv

In [3]:
from google.colab import files
uploaded = files.upload()


Saving zones3DFinal.csv to zones3DFinal.csv


In [4]:
print(uploaded)

{'zones3DFinal.csv': b'IfcSpace/397,-12.7,14.42,0.49,"[3, 0, 1, 2, 3, 3, 4, 5, 3, 3, 5, 1, 3, 6, 1, 0, 3, 6, 3, 1, 3, 7, 8, 4, 3, 7, 4, 3, 3, 9, 7, 3, 3, 10, 11, 8, 3, 10, 8, 7, 3, 12, 11, 10, 3, 13, 14, 12, 3, 13, 12, 10, 3, 15, 14, 13, 3, 16, 15, 13, 3, 17, 18, 15, 3, 17, 15, 16, 3, 19, 20, 6, 3, 19, 6, 0, 3, 0, 2, 21, 3, 0, 21, 19, 3, 22, 21, 2, 3, 22, 2, 1, 3, 1, 5, 23, 3, 1, 23, 22, 3, 23, 5, 4, 3, 24, 23, 4, 3, 24, 4, 8, 3, 24, 8, 25, 3, 26, 25, 8, 3, 26, 8, 11, 3, 27, 26, 11, 3, 27, 11, 12, 3, 28, 27, 12, 3, 28, 12, 14, 3, 29, 28, 14, 3, 29, 14, 15, 3, 30, 29, 15, 3, 30, 15, 18, 3, 18, 17, 31, 3, 18, 31, 30, 3, 32, 31, 17, 3, 32, 17, 16, 3, 33, 32, 16, 3, 33, 16, 13, 3, 10, 33, 13, 3, 10, 34, 33, 3, 7, 34, 10, 3, 7, 35, 34, 3, 35, 7, 9, 3, 35, 9, 36, 3, 37, 36, 9, 3, 37, 9, 3, 3, 3, 6, 20, 3, 3, 20, 37, 3, 22, 19, 21, 3, 20, 19, 22, 3, 37, 20, 22, 3, 23, 37, 22, 3, 24, 37, 23, 3, 35, 36, 37, 3, 35, 37, 24, 3, 25, 35, 24, 3, 34, 35, 25, 3, 26, 34, 25, 3, 27, 33, 34, 3, 27, 34, 26

In [5]:
import io
points_df = pd.read_csv(io.StringIO(uploaded['zones3DFinal.csv'].decode('utf-8')), names= ['name', 'x', 'y', 'z', 'indices'])


In [6]:
print(points_df[112:124])

             name     x      y     z  \
112  IfcSpace/407 -0.94  43.79  4.89   
113  IfcSpace/407 -0.94  43.79  0.49   
114  IfcSpace/407 -4.61  43.79  0.49   
115  IfcSpace/407 -4.61  43.79  4.89   
116  IfcSpace/407 -4.61  40.01  0.49   
117  IfcSpace/407 -4.61  40.01  4.89   
118  IfcSpace/407 -5.91  40.01  0.49   
119  IfcSpace/407 -5.91  40.01  4.89   
120  IfcSpace/407 -5.91  38.57  0.49   
121  IfcSpace/407 -5.91  38.57  4.89   
122  IfcSpace/407 -0.94  38.57  0.49   
123  IfcSpace/407 -0.94  38.57  4.89   

                                               indices  
112  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
113  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
114  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
115  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
116  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
117  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
118  [3, 0, 1, 2, 3, 3, 0, 2, 3, 3, 2, 4, 3, 5, 3, ...  
119  [3, 0, 1, 2, 3, 3,

In [7]:
def from_vertices_to_points_3D(vertices):
  return [[vertex.X(), vertex.Y(), vertex.Z()] for vertex in vertices]

def from_points_to_vertices_3D(points):
  return [Vertex.ByCoordinates(point[0], point[1], point[2]) for point in points]

In [8]:
def make_faces_from_indices(points, face_indices):
  l = len(face_indices)
  i = 0
  triangles = []

  while i < l:
    curr = face_indices[i]
    i+=1
    lis = face_indices[i:curr+i]
    triangles.append(lis)

    i+=curr

  faces_list = []

  for triangle in triangles:
    v1 = points[triangle[0]]
    v2 = points[triangle[1]]
    v3 = points[triangle[2]]
    v_list = from_points_to_vertices_3D([v1, v2, v3])
    f = Face.ByVertices(v_list)
    faces_list.append(f)

  return faces_list



In [11]:
space397 = points_df[0:38]
space397_points = []

for row in space397.itertuples():
  space397_points.append([row[2], row[3], row[4]])

space397_faces_indices = [int(i) for i in space397.iloc[0, 4][1:-1].split(', ')]

fs397 = make_faces_from_indices(points=space397_points, face_indices=space397_faces_indices)
Topology.Show(fs397, renderer='colab', vertexSize=3)

In [12]:
space399 = points_df[74:112]
space399_points = []

for row in space399.itertuples():
  space399_points.append([row[2], row[3], row[4]])

space399_faces_indices = [int(i) for i in space399.iloc[0, 4][1:-1].split(', ')]

fs399 = make_faces_from_indices(points=space399_points, face_indices=space399_faces_indices)
Topology.Show(fs399, renderer='colab', vertexSize=3)

In [13]:
space407 = points_df[112:124]
space407_points = []

for row in space407.itertuples():
  space407_points.append([row[2], row[3], row[4]])

space407_faces_indices = [int(i) for i in space407.iloc[0, 4][1:-1].split(', ')]

fs407 = make_faces_from_indices(points=space407_points, face_indices=space407_faces_indices)
Topology.Show(fs407, renderer='colab', vertexSize=3)

In [14]:
spaceLastPoints = [[-5.455812454223633, 0.2500286102294922, 5.106015682220459],
             [-5.455812454223633, 0.2500286102294922, 0.48601627349853516],
             [-5.455812454223633, 2.169308662414551, 0.48601627349853516],
             [-5.455812454223633, 2.169308662414551, 5.106015682220459],
             [-7.132173538208008, 2.169308662414551, 0.48601627349853516],
             [-7.132173538208008, 2.169308662414551, 5.106015682220459],
             [-7.132173538208008, 0.2500286102294922, 0.48601627349853516],
             [-7.132173538208008, 0.2500286102294922, 5.106015682220459]]

# spaceLastVertices = from_points_to_vertices_3D(spaceLastPoints)

spaceLastFaces = [3, 0, 1, 2,
                  3, 3, 0, 2,
                  3, 3, 2, 4,
                  3, 5, 3, 4,
                  3, 5, 4, 6,
                  3, 7, 5, 6,
                  3, 7, 6, 1,
                  3, 0, 7, 1,
                  3, 2, 1, 4,
                  3, 1, 6, 4,
                  3, 5, 0, 3,
                  3, 5, 7, 0]

fs = make_faces_from_indices(points=spaceLastPoints, face_indices=spaceLastFaces)
Topology.Show(fs, renderer='colab', vertexSize=3)